# TSA Chapter 3: Random Walk Process

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_random_walk/TSA_ch3_random_walk.ipynb)

This notebook demonstrates:
- Random walk simulation: multiple paths, variance growth Var(X_t)=t*sigma^2, and characteristic slow ACF decay.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 500

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# Multiple random walk paths
ax1 = axes[0]
for i in range(10):
    eps = np.random.normal(0, 1, n)
    rw = np.cumsum(eps)
    ax1.plot(rw, linewidth=0.8, alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
ax1.set_title('10 Random Walk Realizations', fontweight='bold')
ax1.set_xlabel('Time (t)')
ax1.set_ylabel(r'$X_t$')

# Variance growth
ax2 = axes[1]
n_sims = 1000
all_rw = np.cumsum(np.random.normal(0, 1, (n_sims, n)), axis=1)
empirical_var = np.var(all_rw, axis=0)
theoretical_var = np.arange(1, n+1)
ax2.plot(empirical_var, color='#1A3A6E', linewidth=1.5, label='Empirical')
ax2.plot(theoretical_var, color='#DC3545', linewidth=1.5, linestyle='--', label=r'Theoretical: $t \cdot \sigma^2$')
ax2.set_title(r'Variance Growth: $Var(X_t) = t \sigma^2$', fontweight='bold')
ax2.set_xlabel('Time (t)')
ax2.set_ylabel('Variance')
ax2.legend()

# ACF of random walk
ax3 = axes[2]
rw_example = np.cumsum(np.random.normal(0, 1, n))
acf_vals = acf(rw_example, nlags=30)
ax3.bar(range(len(acf_vals)), acf_vals, color='#1A3A6E', alpha=0.7)
ax3.axhline(y=1.96/np.sqrt(n), color='red', linestyle='--', linewidth=0.8)
ax3.axhline(y=-1.96/np.sqrt(n), color='red', linestyle='--', linewidth=0.8)
ax3.set_title('ACF of Random Walk (slow decay)', fontweight='bold')
ax3.set_xlabel('Lag')
ax3.set_ylabel('ACF')

plt.tight_layout()
save_chart(fig, 'ch3_random_walk')
plt.show()

print("Random Walk Properties:")
print("  X_t = X_{t-1} + eps_t")
print("  E[X_t] = X_0")
print("  Var(X_t) = t * sigma^2 (grows linearly!)")
print("  ACF(h) approx 1 for all h (non-stationary signature)")